In [3]:
import numpy as np
from scipy.interpolate import interp2d, interpn
from sirius._sirius_utils._math import bilinear_interpolate, interp_array
import matplotlib.pyplot as plt
import ipympl
import xarray as xr
import dask.array as da
import cngi.conversion.convert_ms as convert_ms
import cngi.conversion.convert_image as convert_image
from cngi.dio import read_vis
import numpy as np
from astropy.coordinates import SkyCoord
from sirius import calc_vis
from astropy.wcs import WCS
import time
deg_to_rad = np.pi/180
from sirius import make_ant_sky_jones, simulation
from sirius._sirius_utils._ant_jones_term import _compute_rot_coords
from sirius._sirius_utils._calc_parallactic_angles import _calc_parallactic_angles, _find_optimal_set_angle
from astropy.time import Time
from astropy.coordinates import (EarthLocation, SkyCoord, AltAz, CIRS)
import astropy.units as u
import pkg_resources

In [2]:
x = np.arange(0, 201, 1)

y = np.arange(0, 201, 1)

xx, yy = np.meshgrid(x, y)

z = np.sin(xx**2+(yy+1)**2)

In [3]:
x

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [4]:
%matplotlib ipympl
plt.figure(figsize = (8, 8))
plt.imshow(z)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
#%%timeit
f = interp2d(x, y, z, kind='linear')
test = f(51.5, 51.5)

In [6]:
#%%timeit
test = bilinear_interpolate(z, np.array([51.5]), np.array([51.5]))

In [7]:
%%timeit
f = interp2d(x, y, z, kind='linear')
test = f(51.5, 51.5)

1.84 ms ± 266 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
test = f(51.5, 51.5)

21.5 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%%timeit
test = bilinear_interpolate(z, np.array([51.5]), np.array([51.5]))

7.28 µs ± 814 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
np.allclose(f(45.5, 51.5), bilinear_interpolate(z, np.array([45.5]), np.array([51.5])))

True

In [11]:
f(45.5, 51.5)

array([-0.03263969])

In [12]:
bilinear_interpolate(z, np.array([45.5]), np.array([51.5]))

array([-0.03263969])

### Beam interpolation

In [14]:
interp_array(np.array([z+2j*z, z+1j*z, z]), np.array([2]), np.array([2]), 4, 4)

array([[0.56429666+1.1285933j ],
       [0.56429666+0.56429666j],
       [0.56429666+0.j        ]], dtype=complex64)

In [29]:
n_l = z.shape[0]
n_m = z.shape[1]
coords = np.array([0.5 + n_l//2, 0.5 + n_m//2])
interpn((x, y), z+2j*z, xi = coords, method = "linear")

array([0.56429664+1.12859327j])

In [21]:
np.allclose(interpn((x, y), z+2j*z, xi = coords, method = "linear"), interp_array(np.array([z+2j*z, z+1j*z, z]), np.array([2]), np.array([2]), 4, 4)[0])

True

In [27]:
%%timeit
interpn((x, y), z+2j*z, xi = coords, method = "linear")

461 µs ± 4.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
%%timeit
interp_array(np.array([z+2j*z]), np.array([2]), np.array([2]), 4, 4)

250 µs ± 4.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Testing bessel

In [5]:
from scipy.special import j1

In [ ]:
numba_scipy

In [16]:
%%timeit
j1(3)

838 ns ± 1.96 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
import numba
import numba_scipy.special

In [12]:
@numba.jit(nopython=True,cache=True,nogil=True)
def bessel_test(x):
    return j1(x)

In [15]:
%%timeit
bessel_test(3.0)

264 ns ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
np.allclose(j1(3), bessel_test(3.0))

True

### Testing direction_rotate

In [5]:
from sirius._sirius_utils._direction_rotate import func_R_x, _func_R_x, func_R_y, _func_R_y, func_R_z, _func_R_z, _directional_cosine, _calc_rotation_mats

In [6]:
np.allclose(func_R_x(2.5), _func_R_x(2.5))

True

In [7]:
np.allclose(func_R_y(2.5), _func_R_y(2.5))

True

In [8]:
np.allclose(func_R_z(2.5), _func_R_z(2.5))

True

In [9]:
_directional_cosine((2.1, 3.2))

array([-0.8617374 , -0.50398523, -0.05837414])

In [10]:
_calc_rotation_mats((2.1, 3.2), (1.1, 2.2))

(array([[ 0.54030231,  0.68032627,  0.49520661],
        [ 0.04912015,  0.56199787, -0.82567888],
        [-0.84003609,  0.47044082,  0.27023102]]),
 array([ 0.84003609, -0.47044082,  0.72976898]))

### Testing rot_coords

In [11]:
from sirius._sirius_utils._ant_jones_term import rot_coord, _rot_coord

In [12]:
np.allclose(rot_coord(1, 2, 3), _rot_coord(1, 2, 3))

True

### Testing Sample_j

In [13]:
from sirius.calc_vis import sample_J, sample_J_analytic

In [14]:
bm_J = np.array([[[z, z], [z, z], [z, z]], [[z, z], [z, z], [z, z]]])
bm_pa = np.array([0, 1])
bm_chan = np.array([0, 1, 2])
lmn = _directional_cosine((2.1, 3.2))
freq = 1.1
pa = 0.8
delta_l = 4
delta_m = 4

In [15]:
bm_J.shape #pa, chan, pol, imshape

(2, 3, 2, 201, 201)

In [16]:
sample_J(bm_J, bm_pa, bm_chan, lmn, freq, pa, delta_l, delta_m)

[99.8138916]
[99.83371503]


array([[0.52961546+0.j],
       [0.52961546+0.j]], dtype=complex64)

In [17]:
%%timeit
sample_J(bm_J, bm_pa, bm_chan, lmn, freq, pa, delta_l, delta_m)

[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[99.8138916]
[99.83371503]
[

### Manual check

In [17]:
sample_J(bm_J, bm_pa, bm_chan, lmn, freq, pa, delta_l, delta_m)

[99.8138916]
[99.83371503]


array([[0.52961546+0.j],
       [0.52961546+0.j]], dtype=complex64)

In [18]:
plt.figure(figsize = (8, 8))
plt.imshow(z)
plt.xlim(99, 100)
plt.ylim(99, 100)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Testing apply_primary_beam

In [19]:
from sirius._sirius_utils._apply_primary_beam import apply_casa_airy_pb, _apply_casa_airy_pb, apply_airy_pb, _apply_airy_pb

In [20]:
pb_parms = {"dish_diameter":25, "blockage_diameter":2, "ipower":1}

In [21]:
print(np.allclose(apply_casa_airy_pb(lmn, 1.2e9, pb_parms), _apply_casa_airy_pb(lmn, 1.2e9, 25, 2, 1)))
print(apply_casa_airy_pb(lmn, 1.2e9, pb_parms))
print(_apply_casa_airy_pb(lmn, 1.2e9, 25, 2, 1))

True
-0.00025784633980701515
-0.0002578463398070177


/users/mbarth/sirius/sirius/sirius/_sirius_utils/_apply_primary_beam.py:23: NumbaWarning: Cannot cache compiled function "_apply_casa_airy_pb" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @jit(nopython=True,cache=True,nogil=True)


In [22]:
#Had to change r_grid to r in the function
print(np.allclose(apply_airy_pb(lmn, 1.2e9, pb_parms), _apply_airy_pb(lmn, 1.2e9, 25, 2, 1)))
print(apply_airy_pb(lmn, 1.2e9, pb_parms))
print(_apply_airy_pb(lmn, 1.2e9, 25, 2, 1))

True
-0.00019940730890659765
-0.00019940730890660027


/users/mbarth/sirius/sirius/sirius/_sirius_utils/_apply_primary_beam.py:47: NumbaWarning: Cannot cache compiled function "_apply_airy_pb" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @jit(nopython=True,cache=True,nogil=True)


### Testing Sample_j_analytic

In [23]:
#lmn, freq, dish_diameter, blockage_diameter, ipower 
sample_J_analytic(lmn, 1.2e9, 25, 2, 1, "airy") #Read up on caching...might need to write our own j1

/users/mbarth/sirius/sirius/sirius/calc_vis.py:184: NumbaWarning: Cannot cache compiled function "sample_J_analytic" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @jit(nopython=True,cache=True,nogil=True)


array([-0.00019941,  0.        ,  0.        , -0.00019941])

In [24]:
sample_J_analytic(lmn, 1.2e9, 25, 2, 1, "casa_airy")

array([-0.00025785,  0.        ,  0.        , -0.00025785])

### Testing numba caching

In [14]:
from numba import jit
@jit(nopython=True,cache=True,nogil=True)
def numba_dumb():
    count = 0
    for i in range(1000):
        count += sample_J_analytic(lmn, 1.2e9, 25, 2, 1, "casa_airy")[0]
    return count

In [15]:
%%timeit -n 250
numba_dumb() #Clearly numba is not dumb

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
NameError: name 'sample_J_analytic' is not defined

In [ ]:
%%timeit -n 250
sample_J_analytic(lmn, 1.2e9, 25, 2, 1, "casa_airy")

### Test numba heterogeonous tuple arguments

In [40]:
@jit("int64(int64[:, :], int64[:])", nopython=True,cache=True,nogil=True)
def big_tuple_user(tuple1, indices):
    value1 = 0
    for i in range(len(indices)):
        value1 = value1 + tuple1[indices[i]][0]
    return value1

@jit("int64(int64[:])", nopython=True,cache=True,nogil=True)
def small_tuple_user(tuple1):
    value1 = 0
    value1 = value1 + tuple1[0]
    return value1

@jit(nopython=True,cache=True,nogil=True)
def tuple_tester(evil_tuple, chooser, chooser2):
    test_val = 0
    for i in range(len(chooser)):
        index = chooser[i]
        testing_numba1 = evil_tuple[index]
        if len(evil_tuple[index]) == 1:
            test_val = test_val + small_tuple_user(evil_tuple[index])
        else:
            test_val = test_val + big_tuple_user(evil_tuple[index], chooser2)
    return test_val

In [41]:
evil_tuple = (np.array(((1, 2, 3), (1, 2, 3))), np.array(((1, 2), (1, 2))), np.array((1, )))
print(evil_tuple)

(array([[1, 2, 3],
       [1, 2, 3]]), array([[1, 2],
       [1, 2]]), array([1]))


In [42]:
tuple_tester(evil_tuple, (0, 1, 2), (0, 1))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(Tuple(array(int64, 2d, C), array(int64, 2d, C), array(int64, 1d, C)), int64)
 
There are 22 candidate implementations:
      - Of which 22 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(Tuple(array(int64, 2d, C), array(int64, 2d, C), array(int64, 1d, C)), int64)':
       No match.

During: typing of intrinsic-call at <ipython-input-40-4d2d7f9960ec> (19)

File "<ipython-input-40-4d2d7f9960ec>", line 19:
def tuple_tester(evil_tuple, chooser, chooser2):
    <source elided>
        index = chooser[i]
        testing_numba1 = evil_tuple[index]
        ^


### Comparing old and new sampling funcs

In [29]:
import numpy as np
c = 299792458
#from direction_rotate import _calc_rotation_mats
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp2d, griddata
import matplotlib.pyplot as plt
def coord_to_indx(l,m,cell_size,image_size):
    l_indx = l/cell_size[0] + image_size[0]//2
    m_indx = m/cell_size[1] + image_size[1]//2
    return l_indx, m_indx

In [30]:
import xarray as xr
#    import dask.array as da
#    from cngi.dio import read_vis
#    import numpy as np
#    from astropy.coordinates import SkyCoord
#    import time

deg_to_rad = np.pi/180
from sirius import make_ant_sky_jones, simulation
from sirius._sirius_utils._ant_jones_term import _compute_rot_coords
from sirius._sirius_utils._calc_parallactic_angles import _calc_parallactic_angles, _find_optimal_set_angle
from sirius._sirius_utils._ant_jones_term import _rot_coord
from astropy.time import Time
from astropy.coordinates import (EarthLocation, SkyCoord, AltAz, CIRS)
import astropy.units as u
import pkg_resources
import time

In [37]:
zpc_dir

'/users/mbarth/sirius/sirius/sirius/data/dish_models/data/EVLA_avg_zcoeffs_SBand_lookup.zpc.zarr'

In [6]:
# Get Zernike models from casadata
zpc_dir = '/users/mbarth/sirius/sirius/data/dish_models/data/EVLA_avg_zcoeffs_SBand_lookup.zpc.zarr'#pkg_resources.resource_filename("data", 'data/dish_models/data/EVLA_avg_zcoeffs_SBand_lookup.zpc.zarr')
zpc_xds = xr.open_zarr(zpc_dir)
beam_models = [zpc_xds]
tel_dir = '/users/mbarth/sirius/sirius/data/telescope_layout/data/vla.d.tel.zarr'
tel_xds = xr.open_zarr(tel_dir)
mxds = read_vis('fm_sim_data.vis.zarr/fm_sim_data.vis.zarr')
print(zpc_xds.chan)

ant_pos = mxds.ANTENNA.POSITION.values
beam_model_map = np.zeros(len(ant_pos), dtype=int)
beam_models = [zpc_xds]

<xarray.DataArray 'chan' (chan: 16)>
array([2.052e+09, 2.180e+09, 2.308e+09, 2.436e+09, 2.564e+09, 2.692e+09,
       2.820e+09, 2.948e+09, 3.076e+09, 3.204e+09, 3.332e+09, 3.460e+09,
       3.588e+09, 3.716e+09, 3.844e+09, 3.972e+09])
Coordinates:
  * chan     (chan) float64 2.052e+09 2.18e+09 2.308e+09 ... 3.844e+09 3.972e+09


In [10]:
#zpc_xds.J
zpc_xds #Why no J?

<xarray.Dataset>
Dimensions:    (chan: 16, coef_indx: 66, pol: 4)
Coordinates:
  * chan       (chan) float64 2.052e+09 2.18e+09 ... 3.844e+09 3.972e+09
  * coef_indx  (coef_indx) int64 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65
  * pol        (pol) int64 5 6 7 8
Data variables:
    ETA        (pol, chan, coef_indx) float64 dask.array<chunksize=(4, 16, 66), meta=np.ndarray>
    ZC         (pol, chan, coef_indx) complex128 dask.array<chunksize=(4, 16, 66), meta=np.ndarray>
Attributes:
    conversion_date:  2021-06-10
    dish_diam:        25
    name:             data/EVLA_avg_zcoeffs_SBand_lookup.csv

In [40]:
# Create J_xds from zpc
pb_parms = {}
pb_parms['fov_scaling'] = 15
#pb_parms['fov_scaling'] = 7
#pb_parms['fov_scaling'] = 9
#pb_parms['mueller_selection'] = np.array([0])#np.arange(16) #np.array([0,5,10,15])#np.arange(16)
pb_parms['mueller_selection'] = np.arange(16)
pb_parms['zernike_freq_interp'] = 'nearest'
pb_parms['freq'] = np.array([3.00e+09])
pb_parms['pa'] = np.array([0.0, 0.7323*np.pi/4, 0.9741*np.pi/4, 0.987*np.pi/4, np.pi/4, np.pi/2])
pb_parms['image_size'] = np.array([500,500])
print(zpc_xds.dish_diam,pb_parms['freq'])

# time, ant, chan, pol, l, m
start = time.time()
J_xds = make_ant_sky_jones([zpc_xds],pb_parms) #[None,None,:,:,:,:]
print("Total time", time.time()-start)
pol = 0
print(J_xds)
J_sub = J_xds.J.isel(model=0,pa=0,pol=pol,chan=0)
plt.figure()
plt.imshow(np.abs(J_sub))
J_sub = J_xds.J.isel(model=0,pa=3,pol=pol,chan=0)
plt.figure()
plt.imshow(np.abs(J_sub))
plt.show()

25 [3.e+09]
Total time 0.9565491676330566
<xarray.Dataset>
Dimensions:  (chan: 1, l: 500, m: 500, model: 1, pa: 6, pol: 4)
Coordinates:
  * chan     (chan) float64 3e+09
  * l        (l) float64 0.03657 0.03643 0.03628 ... -0.03614 -0.03628 -0.03643
  * m        (m) float64 -0.03657 -0.03643 -0.03628 ... 0.03614 0.03628 0.03643
  * pa       (pa) float64 0.0 0.5751 0.7651 0.7752 0.7854 1.571
  * pol      (pol) int64 0 1 2 3
Dimensions without coordinates: model
Data variables:
    J        (model, pa, chan, pol, l, m) complex128 (-0.0015533528739918353-...


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
from sirius.calc_vis import beam_models_to_tuple

In [12]:
beam_models_to_tuple(beam_models, beam_model_map)

KeyError: 'pb_func'

### List of questions:

1. Why no J in zpc.xds?

2. Why does sample_j_analytic choose indices 0, 3?

3. Image plane approximation: what happens if two sources are at very different n values?

